In [20]:
import pyxem as pxm
import pymatgen as pmg
import numpy as np
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.utils.plot import generate_marker_inputs_from_peaks
from pyxem.utils.sim_utils import peaks_from_best_template
import hyperspy.api as hs
%matplotlib tk

In [21]:
if False:
    sample = pxm.load('/home/pc494/Documents/data/cBN/160401_MultipleZoneAxes/160401_03.blo')
    sample = sample.inav[50:-60,15:130]

In [22]:
print(sample)
half_side_length = (512/2)

<Signal2D, title: , dimensions: (115, 115|512, 512)>


In [23]:
dp = pxm.ElectronDiffraction(sample.inav[:10,:10])

In [24]:
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]))


A Jupyter Widget

In [25]:
dp.set_calibration(1/half_side_length)

In [26]:
a = 3.615
B = pmg.Element("B")
N = pmg.Element("N")
lattice = pmg.Lattice.cubic(a)
structure = pmg.Structure.from_spacegroup("F-43m",lattice, [B,N], [[0, 0, 0],[0.25,0.25,0.25]])

In [27]:
def build_structure_lib(structure,rot_list):
    struc_lib = dict()
    struc_lib["A"] = (structure,rot_list)
    return struc_lib    

def build_linear_grid_in_euler(alpha_max,beta_max,gamma_max,resolution):
    a = np.arange(0,alpha_max,step=resolution)
    b = np.arange(0,beta_max,step=resolution)
    c = np.arange(0,gamma_max,step=resolution)
    from itertools import product
    return list(product(a,b,c))

In [28]:
edc = pxm.DiffractionGenerator(300, 5e-2)
diff_gen = pxm.DiffractionLibraryGenerator(edc)
rot_list = build_linear_grid_in_euler(90,180,90,5) 
struc_lib = build_structure_lib(structure,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                           calibration=1/half_side_length,
                                           reciprocal_radius=0.6,
                                           half_shape=(half_side_length,half_side_length),
                                           representation='euler',
                                           with_direct_beam=False)

In [29]:
indexer = IndexationGenerator(dp,library)
match_results = indexer.correlate()

A Jupyter Widget

In [32]:
peaks = match_results.map(peaks_from_best_template,phase=["A"],library=library,inplace=False)
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
dp.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    ## THERE IS A GOTCHA HERE DUE TO WEAK REFLECTION
    m = hs.markers.point(x=mx,y=my,color='red',marker='x') #see visual test
    dp.add_marker(m,plot_marker=True,permanent=True)

A Jupyter Widget